### Imports

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import ToTensor
from utils import split_dataset, train_model, plot_metrics

### Initialize Dataset

In [ ]:
# Constants
TEST_DATA_DIR = 'asl-alphabet/asl_alphabet_test/asl_alphabet_test'
TRAIN_DATA_DIR = 'asl-alphabet/asl_alphabet_train/asl_alphabet_train'
SEED = 0

In [ ]:
# Seed PyTorch
torch.manual_seed(SEED)

# Initialize dataset
dataset = torchvision.datasets.ImageFolder(
    root=TRAIN_DATA_DIR,
    transform=ToTensor()
)
num_inputs = np.array(dataset[0][0].numpy().shape).prod()
num_outputs = len(dataset.classes)

# Check for CUDA GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')

### Model #1 - MLP

This simple model is used to verify that the supporting data loading, splitting, training and validation functions are working correctly by training the model on a small subset of the data. The model metrics should show distinct decreasing loss and attain a high training accuracy.

In [ ]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.hidden1 = nn.Linear(num_inputs, 1000)
        self.hidden2 = nn.Linear(1000, 200)
        self.output = nn.Linear(200, num_outputs)

    def forward(self, x):
        x = self.hidden1(x)
        x = F.relu(x)
        x = self.hidden2(x)
        x = F.relu(x)
        x = self.output(x)
        return x

model1 = Net1()
model1 = model1.to(device)

In [ ]:
train_loader1, valid_loader1 = split_dataset(dataset, 58, 0.5, 29, SEED)
train_metrics1, valid_metrics1 = train_model(model1, train_loader1, valid_loader1, 5e-4, 100, device)

In [ ]:
plot_metrics(train_metrics1, valid_metrics1)

### Model #2 - Base CNN

This model is a simple CNN used to verify that such an architecture is able to train on the data and provide baseline performance. The architecture is based on the baseline model used in this [paper](https://ieeexplore.ieee.org/document/8392660) but does not make use of dropout layers or any other form of regularization.

In [ ]:
class Net2(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolutional layer 1
        self.conv1 = nn.Conv2d(3, 32, 5, padding=2)
        self.pool1 = nn.MaxPool2d(2, 2)
        # self.dropout1 = nn.Dropout2d(0.2)

        # Convolutional layer 2
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        # self.dropout2 = nn.Dropout2d(0.2)

        # Fully connected layer 1
        self.fc1 = nn.Linear(32 * 50**2, 128)
        # self.dropout3 = nn.Dropout2d(0.2)

        # Fully connected layer 2
        self.fc2 = nn.Linear(128, 64)
        # self.dropout4 = nn.Dropout2d(0.2)

        # Output layer
        self.fc3 = nn.Linear(64, 29)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model2 = Net2()
model2 = model2.to(device)

In [ ]:
from torchsummary import summary
summary(model2, input_size=(3, 200, 200))

In [ ]:
train_loader2, valid_loader2 = split_dataset(dataset, 1000, 0.8, 100, SEED)
train_metrics2, valid_metrics2 = train_model(model2, train_loader2, valid_loader2, 5e-4, 10, device, conv=True)

In [ ]:
plot_metrics(train_metrics2, valid_metrics2)